In [1]:
#include "Spielbrett.hpp"

In [2]:
SVG brett = SVG(600,600);

In [3]:
//Spieleranzahl erfragen 
int spieleranzahl = 0;
do{
    cout<<"Wie viele Spieler spielen? ";
    cin>>spieleranzahl;
}while(spieleranzahl < 2 || spieleranzahl >4);

Wie viele Spieler spielen? 2


In [4]:
Spielbrett spielbrett = Spielbrett(spieleranzahl, &brett);
list<Spieler*> spieler = spielbrett.getSpielerListe();
Text txt = Text();
string key = "";

In [5]:
//Spielernamen erfragen 
cout << "gebt euch namen!\n"; 
string name = "";
for(Spieler* s: spieler){
    cin >> name; 
    s->setName(name); 
}

gebt euch namen!
a
b


In [6]:
void hypothek(Spieler &s){
    
    AlgoViz::sendText("möchtest du eine Hypothek aufnehmen, auflösen, ein Haus kaufen oder weiter spielen? (1 / 2 / 3 / 4)"); 
    key = brett.lastKey();
    
    
    
    while(key != "4" && key!="3" && key != "1" && key != "2" ){
        key = brett.lastKey();
    }
    
    int num; 
    vector <Straße*> streets = s.getStreets();
    string allStr = "";
    
    if(key == "1"){
        if(streets.size() > 0){
            AlgoViz::sendText("für welche Straße möchtest du eine Hypothek aufnehmen? (0 - n)"); 
            AlgoViz::sleep(3000); 
            
            for(Straße *str: streets){
                cout << str->getName() << "\n"; 
            }
            cout<<"-------------------\n";
        //Damit es zu keiner Exception kommen kann
        do{
            cin >> num; 
        }while(streets.size() < num);
        
        s.HypothekAufnehmen(*streets.at(num));
        }
    }
    
    else if(key=="2"){
        vector <Straße*> streetsHypo = s.getStreetsHypothek();
        if(streetsHypo.size() == 0){
           AlgoViz::sendText("Du hast für keine deiner Straßen bisher eine Hypothek aufgenommen!");  
           AlgoViz::sleep(3000);
        }else{
            
            AlgoViz::sendText("welche deiner Straßen möchtest du zurück kaufen? (0-n)"); 
            AlgoViz::sleep(3000);

            for(Straße *str: streetsHypo){
                cout << str->getName() << "\n";    
                }
            cout<<"-------------------\n";

            //Damit es zu keiner Exception kommen kann
            do{
                cin >> num; 
            }while(streetsHypo.size()< num);

            s.HypothekAufloesen(*streetsHypo.at(num));
            
        }
         
    }else if(key =="3"){
       if(streets.size() > 0){
            AlgoViz::sendText("für welche Straße möchtest du ein Haus kaufen? (0 - n)"); 
            AlgoViz::sleep(3000); 
            
            for(Straße *str: streets){
                cout << str->getName() << "\n"; 
            }
           cout<<"-------------------\n";
            
        //Damit es zu keiner Exception kommen kann
        do{
            cin >> num; 
        }while(streets.size() < num);
        
        if(s.hausbauMoeglich(*streets.at(num))){
            s.buildHouse(*streets.at(num));
    
        }else{
           AlgoViz::sendText("Hausbau für nicht möglich!" );
           AlgoViz::sleep(3000); 
        }
           
     }
    }
}

In [7]:
void kaufen(Spieler &spieler, Straße &straße){
    
    //Funktion zum kaufen einer Straße
    
    AlgoViz::sendText("möchtest du diese Straße für " + to_string(straße.getMiete()) + " kaufen? 'j' / 'n'?");
    key = brett.lastKey(); 
    
    while(key != "j" && key != "n"){
        key = brett.lastKey();
    }
    
    if(key == "j"){   

        if(spieler.kaufenMoeglich(straße)){
            spieler.straßeKaufen(straße);
            AlgoViz::sendText("die Straße wurde erfolgreich gekauft!"); 
            AlgoViz::sleep(3000);
            
        }
        else{
            AlgoViz::sendText("du hast nicht genug Geld um diese Straße zu kaufen");
            AlgoViz::sleep(3000);
            

        }
        
        
    }
}

In [8]:
bool zahlen(Spieler &s, Straße &straße){
    
    
    //zahlen bestimmt nachher spiel und damit den Spielabbruch
    
    bool zahlen = false; 
    
    for(Spieler* s2: spieler){
        if(s2->besitzeStraße(straße)){
            
            AlgoViz::sendText("du bist auf der Straße von " + s2->getName() + " gelandet!");
            AlgoViz::sleep(3000);
            //prüfe ob wir überhaupt Miete zahlen können 
                        
            if(s.mieteZahlenMoeglich(straße)){
                //falls ja:
                
                s.mieteZahlen(*s2, straße);
                int miete = straße.getMiete();

                AlgoViz::sendText("du musstest "+ to_string(miete) +" zahlen");
                AlgoViz::sleep(3000);
            }
            
            //falls nein: 
            else{
                AlgoViz::sendText("du kannst dir die Miete leider nicht leisten - Game Over!");
                AlgoViz::sleep(5000); 
                }
           
            } 
    }
    
    return zahlen;
}

In [9]:
bool spielzug(Spieler &spieler, Spielbrett &spielbrett){
    
    //bool um das Spiel zu beenden
    bool spiel = false;  
    string name = spieler.getName() + " ist am Zug";
    AlgoViz::sendText(name);   
    AlgoViz::sleep(3000);
    
    //mehr Interaktion
    AlgoViz::sendText("um zu würfeln, drücke 'w'!");
    key = brett.lastKey();
    
    while(key != "w"){
        key = brett.lastKey(); 
    }
    
    int wuerfel = spieler.wuerfeln();
    AlgoViz::sendText("du hast eine "+ to_string(wuerfel)+" gewürfelt"); 
    AlgoViz::sleep(2000);
    spielbrett.laufen(spieler, wuerfel); 
    
    Straße* str = spielbrett.getStraße(spieler.getFeldPos()); 
    string street = "du bist auf " + str->getName() + " gelandet!"; 
    AlgoViz::sendText(street);
    AlgoViz::sleep(3000);
    
    bool verkauft = str->getVerkauft();
    if(str->getName() == "PRISON"){
        AlgoViz::sendText("Du musst nächste Runde aussetzen!");
        AlgoViz::sleep(3000);    
    }
    else if(str->getName() =="LOS"){
        AlgoViz::sendText("Du bekommst 100 Algos!");
        AlgoViz::sleep(3000);
    }
    else if(!verkauft){
        //nochmal einzelne Funktionen fürs kaufen und zahlen     
        kaufen(spieler, *str);
    }
    
    else if(!spieler.besitzeStraße(*str) && !str->getHypothek()){
        spiel = zahlen(spieler, *str);
    }
    
    
    
    //Hier hypothek Methode aufrufen, fallls noch nicht GameOver
    if(!spiel && !spieler.getImprisoned()){
        hypothek(spieler);
    }
    //Kapital nochmal ausprinten
    string kapAnz = "neues Kapital von " + (spieler.getName()) + " : " + to_string((spieler.getKapital())) + " Algos";
    Text txt = Text(kapAnz, 200, 250, &brett);
    
    return spiel;
    
}


In [ ]:
    //main programm// 

bool spielEnde = false;
int i = 0;

while(!spielEnde){
    for(Spieler* s: spieler){
        if(!s->getImprisoned()){
            i++;
            string kapAnz = "Kapital von " + (s->getName()) + " : " + to_string((s->getKapital())) + " Algos";
            Text txt = Text(kapAnz, 220, 320, &brett);
            spielEnde = spielzug(*s, spielbrett);
            //ZUM TESTEN
            if(i>10){
                spielEnde = true;
                }
            }
        else{
            s->setImprisoned(false);
            AlgoViz::sendText("du bist im Gefängnis - eine Runde aussetzen!"); 
            AlgoViz::sleep(3000);
        }
       
    }
}

//Ruft Destruktoren für Spieler und Straßen auf
spielbrett.deleteSpielerStraßen();



